## *Understanding Algorand Addresses
#### 03.2 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2023-02-07 (started 2022-01-05)

* The nature and dimension of Algorand addresses
* Encodings: Hex, Base32, Base 64
* How to decode and understand Algorand addresses
* How the mnemonic passphrase works

In [1]:
import base64
import algosdk

## Algorand addresses
* Private key is the basis
* Length is 32 bytes = 256 bits
* How many different addresses are there? Answer: $2^{256}$

In [2]:
combinations = 2**256                     
print(combinations)                     # very hard to read number

115792089237316195423570985008687907853269984665640564039457584007913129639936


In [3]:
f"{combinations:1.3E}"

'1.158E+77'

Compare this to the number of atoms in the observable universe, which is estimated to be between $10^{70}$ and $10^{80}$

#### Cracking Algorand
If you can guess 1 million keys per second, how long would it take to guess all of them?

In [4]:
guess = 1E6
print( f"{combinations/guess :1.3E} seconds" )
print( f"{combinations/guess/60/60/24 :1.3E} days" )
print( f"{combinations/guess/60/60/24/365 :1.3E} years" )

1.158E+71 seconds
1.340E+66 days
3.672E+63 years


**EXERCISE:** 
* How long would it take if our computer were 1000 times faster?
* And if we could use all estimated 5 billion computers in the world?

In [5]:
# Python code goes here

## *Encodings
* A byte can take $2^8 = 256$ different values
    * Decimal representation requires 3 digits
* Encoding = find a way to represent one byte with fewer symbols
* Encodings make a compromise between efficiency and readability
* Below some general examples of decoding and encoding strings and numbers in Python

#### (1) Hex encoding
* Find a way to represent one byte with only 2 symbols
+ Using numbers `0`-`9` plus letters `a`-`f` $\rightarrow$ 16 symbols
+ One symbol encodes $16=2^4$ values $\rightarrow$ 4 bits
+ Two symbols encode $16 \cdot 16 = 2^8 = 256$ values  $\rightarrow$ 8 bits = 1 byte

In [6]:
numbers = [0, 1, 10, 15, 16, 17, 32]
print(numbers)
[hex(number) for number in numbers]

[0, 1, 10, 15, 16, 17, 32]


['0x0', '0x1', '0xa', '0xf', '0x10', '0x11', '0x20']

#### (2) Base 32 encoding
+ Using 32 symbols, *not* standardized
+ One symbol has $32 = 2^5$ possibilities $\rightarrow$ 5 bits
+ Example: letters `A`-`Z` and numbers `2`-`7`   (RFC 4648)
    + When reading the examples below, remember that `A` corresponds to decimal zero


In [7]:
numbers = [0, 1, 25, 26, 31, 32, 1023]
print(numbers)
[base64.b32encode(number.to_bytes(5, 'big')) for number in numbers]

[0, 1, 25, 26, 31, 32, 1023]


[b'AAAAAAAA',
 b'AAAAAAAB',
 b'AAAAAAAZ',
 b'AAAAAAA2',
 b'AAAAAAA7',
 b'AAAAAABA',
 b'AAAAAA77']

#### (3) Base 64 encoding
+ Using 64 symbols, *not* standardized
+ One symbol has $64 = 2^6$ possibilities $\rightarrow$ 6 bits
* Example: letters `A`-`Z`, `a`-`z`, numbers `0`-`9` and special characters `+`, `/`
    * When reading the examples below, remember that `A` corresponds to decimal zero


In [9]:
numbers = [0, 1, 25, 26, 51, 52, 61, 62, 63, 64]
print(numbers)
[base64.standard_b64encode(number.to_bytes(3, 'big')) for number in numbers]

[0, 1, 25, 26, 51, 52, 61, 62, 63, 64]


[b'AAAA',
 b'AAAB',
 b'AAAZ',
 b'AAAa',
 b'AAAz',
 b'AAA0',
 b'AAA9',
 b'AAA+',
 b'AAA/',
 b'AABA']

### Algorand address encoding

In [10]:
# Account info generation.
private_key, public_key = algosdk.account.generate_account()

##### Private key
* Algorand stores the private key (256 bits) together with public key (256 bits) in base64 encoding
    * Hence we need to store 512 bits
* In practise, there are 88 chacters that store 6bits each
* The last two characters are a checksum

In [11]:
print( private_key )
print( len(private_key) )
print( (len(private_key)-2) * 6 )           # 6bits per symbol, two adresses encoded, last two symbols ("==") do not count

cm+7rQKZg8kndU2nfEiHMLdiHkPSRSiY2AAcVAR7rgbnLdgxZZbGnpm3W/db0WHtV5ErDfhaBiCepmELjNDoKw==
88
516


##### Public key
* Algorand stores public key (256 bits) plus a 4 byte checksum (32 bits) in base32 encoding

In [12]:
print( public_key )
print( len(public_key) )
len(public_key) * 5                        # 5bits per symbol, 256+32 = 288

44W5QMLFS3DJ5GNXLP3VXULB5VLZCKYN7BNAMIE6UZQQXDGQ5AVRSVMFAY
58


290

### *Inside the address
* Keys and addresses are actually very long nubers
* Can be represented in several ways

In [13]:
# Standard base32 representation
print(public_key)
print('')

# The decoded address is only 32 Bytes long
decoded_addr = algosdk.encoding.decode_address(public_key)
print(len(decoded_addr))
print('')

# Decimal number
print(int.from_bytes(decoded_addr,byteorder='big'))
print('')

# All 32 bytes of the address
for my_byte in decoded_addr:
    print(f'{my_byte:d}', end=' ')
print('\n')

# All 256 bits of the address
for my_byte in decoded_addr:
    print(f'{my_byte:>08b}', end=' ')
print('\n')

# Hex repsresentation
print(decoded_addr.hex())

44W5QMLFS3DJ5GNXLP3VXULB5VLZCKYN7BNAMIE6UZQQXDGQ5AVRSVMFAY

32

104565268249596423509936335202102886678664209289721279658285888168810664486955

231 45 216 49 101 150 198 158 153 183 91 247 91 209 97 237 87 145 43 13 248 90 6 32 158 166 97 11 140 208 232 43 

11100111 00101101 11011000 00110001 01100101 10010110 11000110 10011110 10011001 10110111 01011011 11110111 01011011 11010001 01100001 11101101 01010111 10010001 00101011 00001101 11111000 01011010 00000110 00100000 10011110 10100110 01100001 00001011 10001100 11010000 11101000 00101011 

e72dd8316596c69e99b75bf75bd161ed57912b0df85a06209ea6610b8cd0e82b


## The Mnemonic key or passphrase
Consists of 25 words, each chose of a list of $2^{11} = 2048$ words. The encoding is simple, we chosse the *n-th* word in the **bip-0039 English word list**<br>
Each word encodes 11 bits, total is

In [14]:
25 * 11

275

24 words would be enough, but again there is 2-byte hash of the private key

In [15]:
25 * 11 - 16

259

#### All 2048 words from the BIP 39  english wordlist
* BIP stands for Bitcoin Improval Proposal
* BIP 39 was proposed in 2013
* BIP 39 has its critics, because there are many variants of the standard, see https://electrum.readthedocs.io/en/latest/seedphrase.html#motivation

**Hint:** Click into the blue bar to hide the output.

In [16]:
print(algosdk.wordlist.word_list_raw())

abandon
ability
able
about
above
absent
absorb
abstract
absurd
abuse
access
accident
account
accuse
achieve
acid
acoustic
acquire
across
act
action
actor
actress
actual
adapt
add
addict
address
adjust
admit
adult
advance
advice
aerobic
affair
afford
afraid
again
age
agent
agree
ahead
aim
air
airport
aisle
alarm
album
alcohol
alert
alien
all
alley
allow
almost
alone
alpha
already
also
alter
always
amateur
amazing
among
amount
amused
analyst
anchor
ancient
anger
angle
angry
animal
ankle
announce
annual
another
answer
antenna
antique
anxiety
any
apart
apology
appear
apple
approve
april
arch
arctic
area
arena
argue
arm
armed
armor
army
around
arrange
arrest
arrive
arrow
art
artefact
artist
artwork
ask
aspect
assault
asset
assist
assume
asthma
athlete
atom
attack
attend
attitude
attract
auction
audit
august
aunt
author
auto
autumn
average
avocado
avoid
awake
aware
away
awesome
awful
awkward
axis
baby
bachelor
bacon
badge
bag
balance
balcony
ball
bamboo
banana
banner
bar
barely
bargain
barre

### *Only the first four letters matter
The function `mnemonic.to_private_key` works with full words or only the first four letters 

#### (a) create private key using full words

In [17]:
passphrase = algosdk.mnemonic.from_private_key(private_key)
print(passphrase)
algosdk.mnemonic.to_private_key(passphrase)

unusual unit fine mother already venue powder estate neutral capable dry indoor shell shy must easy agree give about dove anger paddle cube abandon flower


'cm+7rQKZg8kndU2nfEiHMLdiHkPSRSiY2AAcVAR7rgbnLdgxZZbGnpm3W/db0WHtV5ErDfhaBiCepmELjNDoKw=='

#### (b) create private key using only the first four letters

In [18]:
# A quick function that truncates every word to the first 4 letters (see appendix for details)
def four_letters(string):
    words = string.split(' ')
    short_words=[word[0:4] for word in words]
    return(' '.join(short_words))

In [19]:
# Compare to passphrase above
short_passphrase = four_letters(passphrase)
print(short_passphrase)
algosdk.mnemonic.to_private_key(short_passphrase)   # same as above!

unus unit fine moth alre venu powd esta neut capa dry indo shel shy must easy agre give abou dove ange padd cube aban flow


'cm+7rQKZg8kndU2nfEiHMLdiHkPSRSiY2AAcVAR7rgbnLdgxZZbGnpm3W/db0WHtV5ErDfhaBiCepmELjNDoKw=='

#### A quick note on passphrases
+ Some software requires the entire words

## Appendix: how `four_letters()` works
A step-by-step construction of the function

In [20]:
string = 'Welcome to the WSC blochchain school'

In [21]:
string[0:4]                 # first four letters

'Welc'

In [22]:
string.split(' ')           # split at space --> create a list

['Welcome', 'to', 'the', 'WSC', 'blochchain', 'school']

In [23]:
# List expression
words = string.split(' ')
[word[0:4] for word in words]

['Welc', 'to', 'the', 'WSC', 'bloc', 'scho']

In [24]:
# Join the list to a string again
words = string.split(' ')
short_words=[word[0:4] for word in words]
print(' '.join(short_words))

Welc to the WSC bloc scho


In [25]:
# package as a function
def four_letters(string):
    words = string.split(' ')
    short_words=[word[0:4] for word in words]
    return(' '.join(short_words))

## Appendix: can you create your own Mnemonic?
* Idea: choose words that tell a story for easy memorizing
* Not possible: we would not know how to choose the 25th word correctly
* Also not advisible: "hand picked" words are not sufficiently random and easier to crack